In [ ]:
import pandas as pd
import numpy as np
import spacy
import sys
import warnings

from imp import reload
from gensim.corpora import Dictionary
from importlib import reload
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.metrics import f1_score
from tensorflow.keras.models import load_model

In [ ]:
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

nlp = spacy.load('en')

In [3]:
df1 = pd.read_csv('datasets/labeledTrainData.tsv', delimiter='\t')
df1 = df1.drop(['id'], axis=1)
df1.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
df2 = pd.read_csv('datasets/imdb_master.csv', encoding='latin-1')
df2.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [5]:
df2 = df2.drop(['Unnamed: 0','type','file'],axis=1)
df2.columns = ["review","sentiment"]
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [6]:
df2 = df2[df2.sentiment != 'unsup']
df2['sentiment'] = df2['sentiment'].map({'pos': 1, 'neg': 0})
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [7]:
df = pd.concat([df1, df2]).reset_index(drop=True)
df.head()

,review,sentiment
0,With all this stuff going down at the moment w...,1
1,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,The film starts with a manager (Nicholas Bell)...,0
3,It must be assumed that those who praised this...,0
4,Superbly trashy and wondrously unpretentious 8...,1


In [ ]:
MAX_SEQUENCE_LEN = 130
UNK = 'UNK'
PAD = 'PAD'

def text_to_id_list(text, dictionary):
    return [dictionary.token2id.get(tok, dictionary.token2id.get(UNK))
            for tok in text_to_tokens(text)]

def texts_to_input(texts, dictionary):
    return sequence.pad_sequences(
        list(map(lambda x: text_to_id_list(x, dictionary), texts)), maxlen=MAX_SEQUENCE_LEN,
        padding='post', truncating='post', value=dictionary.token2id.get(PAD))

def text_to_tokens(text):
    return [tok.text.lower() for tok in nlp.tokenizer(text)
            if not (tok.is_punct or tok.is_quote)]

def build_dictionary(texts):
    d = Dictionary(text_to_tokens(t)for t in texts)
    d.filter_extremes(no_below=3, no_above=1)
    d.add_documents([[UNK, PAD]])
    d.compactify()
    return d

In [ ]:
dictionary = build_dictionary(df.review)

In [ ]:
dictionary.save('dictionary-sentiment')

In [12]:
loaded_dict = Dictionary.load('dictionary-sentiment')

len(loaded_dict)

69763

In [14]:
x_train = texts_to_input(df.review, dictionary)
x_train

array([[  224,    12,   199, ...,   134,   102,    18],
       [  230,   238,   295, ...,   262,   279,   105],
       [  193,    75,   184, ...,   474,   475,    16],
       ...,
       [   99,  2113,  6024, ...,   237,     8,   416],
       [    8,  4453,  1455, ...,  1048,   809,   181],
       [  226,  3032,  2740, ..., 36552,    16,   193]], dtype=int32)

In [15]:
y_train = df['sentiment']
y_train

0        1
1        1
2        0
3        0
4        1
        ..
74995    1
74996    1
74997    1
74998    1
74999    1
Name: sentiment, Length: 75000, dtype: int64

In [17]:
y_train = np.asarray(y_train)
y_train

array([1, 1, 0, ..., 1, 1, 1])

In [18]:
max_features = 6000

maxlen = 130

embed_size = 128
model = Sequential()
model.add(Embedding(len(dictionary), embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 60000 samples, validate on 15000 samples
Epoch 1/3
60000/60000 [==============================] - 271s 5ms/sample - loss: 0.3670 - acc: 0.8293 - val_loss: 0.2173 - val_acc: 0.9251
Epoch 2/3
60000/60000 [==============================] - 271s 5ms/sample - loss: 0.1643 - acc: 0.9402 - val_loss: 0.1007 - val_acc: 0.9669
Epoch 3/3
60000/60000 [==

In [19]:
df_test = pd.read_csv('datasets/testData.tsv', header=0, delimiter='\t', quoting=3)
df_test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [22]:
df_test["sentiment"] = df_test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
df_test.head()

,id,review,sentiment
0,"""12311_10""","""Naturally in a film who's main themes are of ...",1
1,"""8348_2""","""This movie is a disaster within a disaster fi...",0
2,"""5828_4""","""All in all, this is a movie for kids. We saw ...",0
3,"""7186_2""","""Afraid of the Dark left me with the impressio...",0
4,"""12128_7""","""A very accurate depiction of small time mob l...",1


In [ ]:
y_test = df_test["sentiment"]
x_test = texts_to_input(df_test.review, dictionary)

In [ ]:
prediction = model.predict(x_test)

In [ ]:
y_pred = (prediction > 0.5)

In [28]:
from sklearn.metrics import f1_score

print('F1-score: {0}'.format(f1_score(y_pred, y_test)))

F1-score: 0.9861099959855479


In [ ]:
model.save('sentiment_model-v2.h5')